In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
 
class Data:
    def __init__(self, DropCol = None, EncConfig = None): #'OneHot' -> all categorical columns with One Hot; 'Num' -> all categorical columns  0,1,2..; Dict_type -> writing which columns how
        self.DataName = 'in-vehicle-coupon-recommendation.csv'
        self.Data = None
        
        self.DecColumn = None
        self.Columns = None
        self.CatColumns = None

        self.DropColumns = DropCol
        self.EncConfiguration = EncConfig
    
    def setSetting(self, data):
        self.Columns = data.columns.tolist()
        self.CatColumns = data.select_dtypes(include=['object']).columns.tolist()

        if self.EncConfiguration is None:
            return True
        elif self.EncConfiguration == 'OneHot':
            self.EncConfiguration = {c: 'OneHot' for c in self.Columns}
            return True
        elif self.EncConfiguration == 'Num':
            self.EncConfiguration = {c: 'Num' for c in self.Columns}
            return True
        elif isinstance(self.EncConfiguration, dict):
            self.EncConfiguration = {c: self.EncConfiguration[c] if c in self.EncConfiguration.keys() else None  for c in self.Columns}
            return True
        else:
            print('Error: Wrong insterted type!')
            return False

    def DropCol(self, data):
        for c in self.DropColumns:
            data = data.drop(c, axis = 1)
        return data
    
    def NanPrepocessing(self, data):
        data = data.drop('car', axis = 1)
        # NanRow = data[data.isna().any(axis=1)]
        data = data.dropna()
        data = data.reset_index(drop=True)
        return data

    def SepDecision(self, data):
        self.DecColumn = data['Y']
        data = data.drop('Y', axis = 1)
        return data
    
    def Encoding(self, data):
        for c in self.CatColumns:
            if self.EncConfiguration[c] == 'OneHot':
                data[c] = self.OneHot(data[[c]])
            elif self.EncConfiguration[c] == 'Num':
                data[c] = self.Num(data[c])
        return data

    def OneHot(self, Col):
        Encoder = OneHotEncoder(drop = 'first', sparse_output=False)
        EncCol = Encoder.fit_transform(Col)
        EncCol_str = []
        for e in EncCol:
            EncCol_str.append(str(e))
        return EncCol_str

    def Num(self, Col):
        UnqCol = Col.unique().tolist()
        EncDict = {c: UnqCol.index(c) for c in UnqCol}
        EncCol = [EncDict[c] for c in Col]
        return EncCol
            
    def fit(self):
        data = pd.read_csv(self.DataName)
              
        if self.DropColumns is not None: 
            data = self.DropCol(data)
        
        data = self.NanPrepocessing(data)

        data = self.SepDecision(data)

        if not self.setSetting(data): return None

        if self.EncConfiguration is not None: 
            data = self.Encoding(data)

        data['Y'] = self.DecColumn
        self.Data = data
        return data

In [2]:
d = Data(DropCol = ['destination'], EncConfig = {'passanger': 'OneHot', 'weather': 'Num'})

In [3]:
data = d.fit()
data

,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,has_children,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,[0. 0. 0.],0,55,2PM,Restaurant(<20),1d,Male,21,Single,0,...,less1,4~8,4~8,less1,1,0,0,0,1,1
1,[1. 0. 0.],0,80,10AM,Coffee House,2h,Male,21,Single,0,...,less1,4~8,4~8,less1,1,0,0,0,1,0
2,[1. 0. 0.],0,80,10AM,Bar,1d,Male,21,Single,0,...,less1,4~8,4~8,less1,1,0,0,0,1,1
3,[1. 0. 0.],0,80,10AM,Carry out & Take away,2h,Male,21,Single,0,...,less1,4~8,4~8,less1,1,1,0,0,1,0
4,[1. 0. 0.],0,80,2PM,Coffee House,1d,Male,21,Single,0,...,less1,4~8,4~8,less1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12074,[0. 0. 1.],1,55,6PM,Carry out & Take away,1d,Male,26,Single,0,...,never,1~3,4~8,1~3,1,0,0,1,0,1
12075,[0. 0. 0.],1,55,7AM,Carry out & Take away,1d,Male,26,Single,0,...,never,1~3,4~8,1~3,1,0,0,0,1,1
12076,[0. 0. 0.],2,30,7AM,Coffee House,1d,Male,26,Single,0,...,never,1~3,4~8,1~3,1,0,0,1,0,0
12077,[0. 0. 0.],2,30,7AM,Bar,1d,Male,26,Single,0,...,never,1~3,4~8,1~3,1,1,1,0,1,0
